<a href="https://colab.research.google.com/github/futugyou/pyproject/blob/master/transformers_demo/text_generation_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required dependencies
%pip install langchain-community langchain-google-genai langchain google-generativeai
# Goal: Generate title, characters, story outline

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", api_key=userdata.get("GOOGLE_API_KEY")
)

In [ ]:
from langchain.schema.runnable import RunnableLambda, RunnableMap, RunnablePassthrough


def make_sequential_chain(steps, input_key="summary"):
    """
    steps: [("title", title_prompt), ("character", character_prompt), ...]
    """
    chain = {"__input__": RunnablePassthrough()}
    for key, prompt in steps:
        chain[key] = (
            {**chain, input_key: chain["__input__"]}
            | prompt
            | llm
            | RunnableLambda(lambda x, k=key: {k: x.content})
        )
    return RunnableMap(
        {input_key: chain["__input__"], **{k: chain[k] for k, _ in steps}}
    )

In [ ]:
# 1. generate title
from langchain import PromptTemplate, LLMChain

template = """<s><|user|>
create a title for a story about {summary}. only return the title.<|end|>
<|assistant|>
"""

title_prompt = PromptTemplate(template=template, input_variables=["summary"])

In [ ]:
# 2. generate characters
template = """<s><|user|>
deacribe the main character of a story about {summary} with the title {title}.
use olny two sentences.<|end|>
<|assistant|>
"""

character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)

In [ ]:
# 3. generate story
template = """<s><|user|>
create a story about {summary} with the title {title}. the main character is:
{character}. only return the story and it cannot be longer than one paragraph.
<|end|>
<|assistant|>
"""

story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)

In [ ]:
steps = [
    ("title", title_prompt),
    ("character", character_prompt),
    ("story", story_prompt),
]
chain = make_sequential_chain(steps)
response = chain.invoke({"summary": "a girl that lost her mother"})
print(response)